In [45]:
#r "nuget:Microsoft.ML.AutoML, 0.17.0-preview"

In [81]:
// =========== PackageHelper Code =========================================
// This code helps with the compilation of scripts.
//
//  Add this code to your C# script before compiling the target assembly
//  Compile the target assembly

using Microsoft.DotNet.Interactive;

using System.IO;
using System.Collections.Generic;
using System.IO.Compression;

#r "nuget:FSharp.Core"
#r "nuget:FSharp.Compiler.Private.Scripting"

static class PackagesHelper
{
    static DependencyProvider _dependencies;

    static string GetPathRelativeToPackages(string file, DirectoryInfo directory)
    {
        var fileName = file.Replace("/", @"\");
        var strippedRoot = fileName.Replace(directory.ToString().Replace("/", @"\"), "");
        var relativePath = Path.Combine(Trim(directory.Parent.Name), Trim(directory.Name), Trim(strippedRoot));
        return relativePath;

        string Trim(string path)
        {
            return path.Trim(Path.DirectorySeparatorChar, Path.AltDirectorySeparatorChar);
        }
    }

    static void ZipInsertFileFrom(ZipArchiveEntry entry, FileInfo file)
    {
        using (StreamWriter writer = new StreamWriter(entry.Open()))
        {
            using (StreamReader reader = new StreamReader(file.Open(FileMode.Open)))
            {
                while(!reader.EndOfStream)
                {
                    var b = reader.Read();
                    writer.Write(b);
                }
            }
        }
    }

    public static FileInfo MakePackageZipFile(FileInfo zipPath, IEnumerable<DirectoryInfo> directories)
    {
        using (FileStream zipToOpen = new FileStream(zipPath.ToString(), FileMode.Create))
        {
            using (ZipArchive archive = new ZipArchive(zipToOpen, ZipArchiveMode.Create))
            {
                foreach (var directory in directories)
                {
                    foreach (var file in directory.EnumerateFiles("*", SearchOption.AllDirectories))
                    {
                        // Strip off and make path relative
                        var nugetPath = GetPathRelativeToPackages(file.FullName, directory);
                        ZipArchiveEntry entry = archive.CreateEntry(nugetPath);
                        ZipInsertFileFrom(entry, file);
                    }
                }
            }
        }
        return zipPath;
    }
    
    public static FileInfo ZipPackages(DirectoryInfo workDir)
    {
        var restoreContext = KernelInvocationContext.Current.HandlingKernel as IPackageRestoreContext;
        var directories = restoreContext.ResolvedPackageReferences.Select(r => r.PackageRoot);
        return MakePackageZipFile(new FileInfo(Path.Combine(workDir.ToString(), "packages.zip")), directories);
    }

    public static void UnzipPackages(FileInfo zipFile, DirectoryInfo extractPath)
    {
         ZipFile.ExtractToDirectory(zipFile.ToString(), extractPath.ToString());
    }
    
    // This returns the full paths to the packagemanager library to be deployed with compiler scripts
    // HardCoded to:
    //     FSharp.Core.dll
    //     Interactive.DependencyManager.dll   ---> this name will definitely change. to Microsoft.DotNet.DependencyManager.dll
    public static IEnumerable<FileInfo> GetPathsToPackageManagerLibraries()
    {
        var restoreContext = KernelInvocationContext.Current.HandlingKernel as IPackageRestoreContext;
        var packages = restoreContext.ResolvedPackageReferences;
        foreach (var package in packages)
        {
            foreach (var path in package.AssemblyPaths)
            {
                var p = path.ToString();
                if (p.EndsWith("FSharp.Core.dll")) yield return path;
                else if (p.EndsWith("Microsoft.DotNet.DependencyManager.dll")) yield return path;
                else if (p.EndsWith("Interactive.DependencyManager.dll")) yield return path;
            }
        }
    }

    // This returns the relative paths to the referenced dll's
    public static IEnumerable<string> GetAssemblyProbingPaths()
    {
        var restoreContext = KernelInvocationContext.Current.HandlingKernel as IPackageRestoreContext;
        var packages = restoreContext.ResolvedPackageReferences;
        foreach (var package in packages)
        {
            foreach (var path in package.AssemblyPaths)
            {
                yield return GetPathRelativeToPackages(path.FullName, package.PackageRoot);
            }
        }
    }

    // This returns the full paths to the referenced dll's
    public static IEnumerable<string> GetNativeProbingPaths()
    {
        var restoreContext = KernelInvocationContext.Current.HandlingKernel as IPackageRestoreContext;
        var packages = restoreContext.ResolvedPackageReferences;
        foreach (var package in packages)
        {
            foreach (var path in package.ProbingPaths)
            {
                yield return GetPathRelativeToPackages(path.FullName, package.PackageRoot);
            }
        }
    }

    // PackageRoot is the directory that the zipfile was unpacked to
    public static void ResolutionHandler(DirectoryInfo packageRoot)
    {
        // Read probees file from local directory
        var nativeProbes = File.ReadAllLines("nativeProbes.probe");
        var assemblyProbes = File.ReadAllLines("assemblyProbes.probe");

        _dependencies = new DependencyProvider(AssemblyProbingPaths, NativeProbingRoots);

        IEnumerable<string> AssemblyProbingPaths()
        {
            foreach (var path in assemblyProbes)
                yield return Path.Combine(packageRoot.FullName, path);
        }

        IEnumerable<string> NativeProbingRoots ()
        {
            foreach (var path in nativeProbes)
            {
                yield return Path.Combine(packageRoot.FullName, path);
            }
        }
    }
}

In [82]:
// =========== Initialize Dll Resolution handlers Code =========================================
// The preparations necessary for this are:
//  1:  Deploy the built assembly to it's target location
//  2:  Deploy the assemblies found using PackagesHelper.GetPathsToPackageManagerLibraries() to the same directory
//  3:  Deploy the text files created by:
//               PackagesHelper.GetAssemblyProbingPaths() and PackagesHelper.GetNativeProbingPaths()
//               to the same directory.
//  4:  Unzip the .zip file created with PackagesHelper.ZipPackages(workDir) to where you want the packages to be stored
//  5:  Add this to the script to be compiled.
//  6:  Invoke DeployedPackageHelper.ResolutionHandler(with the path to the ZipRootFolder)

// Initialize assembly resolution handler
using System.IO;
var FileInfo ResolutionHandler;

PackagesHelper.ResolutionHandler(packageRoot);

In [82]:
// Preparation with compilation
//  1:  Inject the code in the two cells above into the script.
//  2:  Initialize Dll Resolution handlers Code should execute first in the compiled script, so I guess it needs to be injected at the versy start
//  3:  The PackageHelper Code is to be used by the Compilation script, it generates a zip file, two text files with reolution information, and a list of dll's 
//      that need to be copied

var workDir = new DirectoryInfo(Path.Combine(Path.GetTempPath(), Path.GetRandomFileName()));
if (!workDir.Exists) workDir.Create();

var zipFile = PackagesHelper.ZipPackages(workDir);

var dllPaths = PackagesHelper.GetPathsToPackageManagerLibraries();

// Add these to deployment

// Assembly probing paths
var assemblyProbingPathsFile = new FileInfo(Path.Combine(workDir.FullName, "assemblyProbes.probe"));
var assemblyProbingPaths = PackagesHelper.GetAssemblyProbingPaths();
File.WriteAllLines(assemblyProbingPathsFile.FullName, assemblyProbingPaths);

// Native probing paths
var nativeProbingPathsFile = new FileInfo(Path.Combine(workDir.FullName, "nativeProbes.probe"));
var nativeProbingPaths = PackagesHelper.GetNativeProbingPaths();
File.WriteAllLines(nativeProbingPathsFile.FullName, nativeProbingPaths);

Console.WriteLine($"{workDir}");

C:\Users\kevinr\AppData\Local\Temp\eyimfeyg.tvs


In [ ]:
// The preparations necessary for this are:
//  1:  Deploy the built assembly to it's target location
//  2:  Deploy the assemblies found using PackagesHelper.GetPathsToPackageManagerLibraries() to the same directory
//  3:  Deploy the text files created by:
//               PackagesHelper.GetAssemblyProbingPaths() and PackagesHelper.GetNativeProbingPaths()
//               to the same directory.
//  4:  Unzip the .zip file created with PackagesHelper.ZipPackages(workDir) to where you want the packages to be stored
//  5:  Add this to the script to be compiled.
//  6:  Invoke DeployedPackageHelper.ResolutionHandler(with the path to the ZipRootFolder)

// Initialize assembly resolution handler
using System.IO;
var FileInfo ResolutionHandler;

PackagesHelper.ResolutionHandler(packageRoot);